## Cleaning Data

In [144]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'C:\Users\Haley\OneDrive\Desktop\FPDS Sample Report.csv') # change file location and file name

In [145]:
# concatenate title and product description
df['title_desc'] = df['Description of Requirement'] + ' '+ df['Product or Service Description']
df = df.astype({"title_desc":'str', "Description of Requirement":'str', 'Product or Service Description': 'str', 'Product or Service Code':'str'}) # changing data types so clean_text function works properly
df.dropna(subset=['Product or Service Code'], inplace=True) # removed rows with null values in 'PSC' column, change later to product_code
df.isnull().sum()

PIID                                                   0
Prepared Date                                          0
Date Signed                                            0
Contracting Office Region                              0
Modification Number                                    0
Period of Performance Start Date                       0
Funding Office ID                                      0
Principal Place of Performance Country Name            0
Vendor Name                                            0
Referenced  IDV PIID                                 328
Referenced IDV Mod Number                            319
(Last Date To Order)                                 555
Purchase Card as Payment Method Code                   0
Purchase Card as Payment Method Description            0
Description of Requirement                             0
IDV NAICS Code                                       328
IDV NAICS Description                                328
NAICS Code                     

In [246]:
# import libraries
import nltk
import re
import string as st
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
default_stopwords = stopwords.words('english') # or any other list of your choice

words = set(nltk.corpus.words.words())

# create clean_text function:
def clean_text(text):

    def remove_punct(text):
        return ("".join([ch for ch in text if ch not in st.punctuation]))
    
    def tokenize(text):
        text = re.split('\s+' ,text)
        return [x.lower() for x in text]
    
    def remove_non_english_words(text):
        return ("".join([w for w in nltk.wordpunct_tokenize(text)]))

    def stemming(text):
        ps = PorterStemmer()
        return [ps.stem(word) for word in text]

    def lemmatize(text):
        word_net = WordNetLemmatizer()
        return [word_net.lemmatize(word) for word in text]

    text = remove_punct(text) # remove punctuation
    text = tokenize(text) # tokenize
    text = remove_stopwords(text) # remove stopwords
    text = stemming(text) # stemming
    text = lemmatize(text) # lemmatization
    
    return text

In [248]:
df['clean_title'] = df['Description of Requirement'].apply(lambda x : clean_text(x))

In [148]:
df['clean_desc'] = df['Product or Service Description'].apply(lambda x : clean_text(x))

In [149]:
#df['clean_title_desc'] = df['title_desc'].apply(lambda x : clean_text(x))

In [150]:
def return_sentences(tokens):
    return " ".join([word for word in tokens])

In [151]:
df['clean_title'] = df['clean_title'].apply(lambda x : return_sentences(x))

In [152]:
df['clean_desc'] = df['clean_desc'].apply(lambda x : return_sentences(x))

In [153]:
df.to_csv('cleaned_data.csv')

In [51]:
few_title = df.clean_title.value_counts()[df.clean_title.value_counts() == 1]
(few_title.count()/df.clean_title.count())*100 # percentage of data that only has 1 instance of a product title
print(few_title)

icasstoner cartridge laserjet printer m750 p3015                                                             1
fac cmr bat survey                                                                                           1
airport transfer service incoming officer family ac                                                          1
book printing right fee                                                                                      1
singso dp packout fmo american officer family mover                                                          1
                                                                                                            ..
operation fuel supply                                                                                        1
annual developmental corrective maintenance service contract energy monitoring system currently installed    1
trash removal dumpster replacement september 2020                                                            1
b

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
plt.cla()
plt.close()
#plt.figure(figsize=(8,8))
filtered_freq.plot(kind = 'barh')
plt.title('Product Titles')
plt.legend('')
plt.xlabel('Frequency')

## Record Linkage

In [171]:
df['clean_title'] = df['clean_title'].astype(str) 
import recordlinkage
indexer = recordlinkage.FullIndex()
pairs = indexer.index(df)

In [173]:
indexer = recordlinkage.BlockIndex(on = 'clean_title')
pairs = indexer.index(df)

In [174]:
len(pairs)

1626

In [235]:
compare_cl = recordlinkage.Compare()
compare_cl.string('clean_title', 'clean_title', method = 'damerau_levenshtein', threshold = 0.7, label = 'clean_title')
compare_cl.string('clean_desc', 'clean_desc', method = 'damerau_levenshtein', threshold = 0.7, label = 'clean_desc')
#compare_cl.exact('Product or Service Code', 'Product or Service Code', label = 'code')

<Compare>

In [236]:
features = compare_cl.compute(pairs, df)

In [237]:
matches = features[features.sum(axis=1) >= 2 ]
matches.head()

clean_title  clean_desc
5   4            1.0         1.0
6   4            1.0         1.0
    5            1.0         1.0
8   7            1.0         1.0
10  9            1.0         1.0
...              ...         ...
548 547          1.0         1.0
549 546          1.0         1.0
    547          1.0         1.0
    548          1.0         1.0
553 552          1.0         1.0

[1542 rows x 2 columns]

In [232]:
ecm = recordlinkage.ECMClassifier()
matches = ecm.fit_predict(features)
matches

MultiIndex([(  5,   4),
            (  6,   4),
            (  6,   5),
            (  8,   7),
            ( 10,   9),
            ( 12,  11),
            ( 14,  13),
            ( 17,  15),
            ( 20,  15),
            ( 20,  17),
            ...
            (537, 536),
            (538, 536),
            (538, 537),
            (547, 546),
            (548, 546),
            (548, 547),
            (549, 546),
            (549, 547),
            (549, 548),
            (553, 552)],
           length=1541)

In [234]:
df.loc[[71], ['clean_title', 'clean_desc']], df.loc[[51], ['clean_title', 'clean_desc']], df.loc[[65], ['clean_title', 'clean_desc']], df.loc[[70], ['clean_title', 'clean_desc']]

(               clean_title               clean_desc
 71  electron comput compon  inform technolog compon,
                clean_title               clean_desc
 51  electron comput compon  inform technolog compon,
                clean_title               clean_desc
 65  electron comput compon  inform technolog compon,
                clean_title               clean_desc
 70  electron comput compon  inform technolog compon)

In [233]:
for index, match in enumerate(matches):
    print(index, match)

0 (5, 4)
1 (6, 4)
2 (6, 5)
3 (8, 7)
4 (10, 9)
5 (12, 11)
6 (14, 13)
7 (17, 15)
8 (20, 15)
9 (20, 17)
10 (23, 22)
11 (29, 28)
12 (30, 28)
13 (30, 29)
14 (31, 28)
15 (31, 29)
16 (31, 30)
17 (34, 33)
18 (36, 35)
19 (44, 43)
20 (49, 48)
21 (51, 50)
22 (65, 50)
23 (65, 51)
24 (66, 50)
25 (66, 51)
26 (66, 65)
27 (68, 50)
28 (68, 51)
29 (68, 65)
30 (68, 66)
31 (69, 50)
32 (69, 51)
33 (69, 65)
34 (69, 66)
35 (69, 68)
36 (70, 50)
37 (70, 51)
38 (70, 65)
39 (70, 66)
40 (70, 68)
41 (70, 69)
42 (71, 50)
43 (71, 51)
44 (71, 65)
45 (71, 66)
46 (71, 68)
47 (71, 69)
48 (71, 70)
49 (73, 50)
50 (73, 51)
51 (73, 65)
52 (73, 66)
53 (73, 68)
54 (73, 69)
55 (73, 70)
56 (73, 71)
57 (74, 50)
58 (74, 51)
59 (74, 65)
60 (74, 66)
61 (74, 68)
62 (74, 69)
63 (74, 70)
64 (74, 71)
65 (74, 73)
66 (75, 50)
67 (75, 51)
68 (75, 65)
69 (75, 66)
70 (75, 68)
71 (75, 69)
72 (75, 70)
73 (75, 71)
74 (75, 73)
75 (75, 74)
76 (76, 50)
77 (76, 51)
78 (76, 65)
79 (76, 66)
80 (76, 68)
81 (76, 69)
82 (76, 70)
83 (76, 71)
84 (76, 73)